In [1]:

import numpy as np
from scipy.linalg import qr, block_diag

def tsqr(A):
    m, n = A.shape  # rows, columns = matrix.shape
    block_size = m // 4 
    num_blocks = 4  # number of blocks based on block size

    # Split matrix A into row-wise blocks
    A_blocks = [A[i * block_size:min((i + 1) * block_size, m), :] for i in range(num_blocks)]

    Q_blocks = []
    R_blocks = []

    # Stage 1: Perform local QR factorization for each block
    for block in A_blocks:
        Q_block, R_block = qr(block, mode="economic")
        Q_blocks.append(Q_block)
        R_blocks.append(R_block)
    # Create block diagonal matrix of first Q matrices
    bigq_block_diag = block_diag(*Q_blocks)
    # Stage 2: Perform QR on stacked R-blocks
    R_stack = np.vstack(R_blocks)
    R_stack1 = np.vstack([R_blocks[0], R_blocks[1]])
    R_stack2 = np.vstack([R_blocks[2], R_blocks[3]])
    Q_01, R_01 = qr(R_stack1, mode="economic")
    Q_11, R_11 = qr(R_stack2, mode="economic")
    stage2qblockdiag = block_diag(Q_01, Q_11)
    # Stage 3
    stage3R = np.vstack([R_01, R_11])
    Q_02, R_02 = qr(stage3R, mode="economic")
    finalQ = bigq_block_diag @ stage2qblockdiag @ Q_02
    finalR = R_02
    return finalQ, finalR
    
    # Example usage 

m=2000
n=20
A = np.random.rand(m,n) # Random matrix (tall and skinny)
CASQR = tsqr(A)
Q, R = CASQR
print("Q Matrix:")
print(Q)
print("\nR Matrix:")
print(R)
# Check if Q is orthogonal and QR = A
if np.allclose(Q.T @ Q, np.eye(n)):
    print("Q is orthogonal")
else:
    print("ERROR: Q is NOT orthogonal")
if np.allclose(A, Q@R):
    print("QR decomposition successful")
else:
    print("QR decomposition unsuccessful")

Q Matrix:
[[-0.01764939  0.0195332   0.01143291 ...  0.00962068 -0.01882444
  -0.01200934]
 [-0.03150051 -0.01995663 -0.01078368 ... -0.02218443 -0.03272062
   0.02209093]
 [-0.01350675 -0.03157139 -0.007325   ...  0.03572674  0.00751247
  -0.00916259]
 ...
 [-0.00016356 -0.00856011 -0.00399551 ...  0.02669546 -0.00088918
  -0.02059238]
 [-0.03889941  0.0328394   0.03071682 ...  0.01483223  0.01938416
   0.01871036]
 [-0.03504168 -0.00868379  0.02024604 ...  0.03032701  0.00678983
   0.02499561]]

R Matrix:
[[-25.67564513 -19.37100435 -19.36311801 -18.76107481 -18.93095659
  -19.3974534  -19.36580939 -19.13378427 -19.58113671 -19.20184321
  -19.43218813 -19.17746607 -18.97179026 -19.48437569 -19.83679338
  -18.89567328 -19.19629728 -18.79601158 -19.12643193 -19.30251168]
 [  0.         -17.33316915  -7.62456434  -7.70856879  -7.76975022
   -7.69431232  -7.75003592  -7.497318    -7.55287494  -7.63141484
   -7.68293842  -7.52670586  -7.9616358   -7.50175656  -7.54501004
   -7.45469316  -